In [1]:
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784', version=1)

In [3]:
from sklearn.model_selection import train_test_split

# Split into Train / Test Set
mnist.keys()
X = mnist.data
y = mnist.target

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.4, test_size=0.2)

In [42]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression

classifiers = [
    ('Random Forest', RandomForestClassifier(n_jobs=-1)),
    ('Neural Network MLP', MLPClassifier()),
    ('Extra Trees', ExtraTreesClassifier(n_jobs=-1)),
    ('Bagging Classifier', BaggingClassifier(n_jobs=-1)),
#     ('Logistic Regression', LogisticRegression())
]

for name, clf in classifiers:
    print(f'**** Cross Validating {name} on Training Set ****')
    scores = cross_val_score(clf, X_train, y_train, cv=3)
    print(scores.mean(), '\n')

**** Cross Validating Random Forest on Training Set ****
0.959357041991758 

**** Cross Validating Neural Network MLP on Training Set ****
0.9449998608340514 

**** Cross Validating Extra Trees on Training Set ****
0.9626071006714229 

**** Cross Validating Bagging Classifier on Training Set ****
0.9243212587415788 

**** Cross Validating Logistic Regression on Training Set ****


/home/nakamuta/Documents/Personal/HandsOnMl/venv/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/home/nakamuta/Documents/Personal/HandsOnMl/venv/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    ht

0.904214067365487 



/home/nakamuta/Documents/Personal/HandsOnMl/venv/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [44]:
from sklearn.ensemble import VotingClassifier

classifiers = [
    ('Random Forest', RandomForestClassifier(n_jobs=-1)),
    ('Neural Network MLP', MLPClassifier()),
    ('Extra Trees', ExtraTreesClassifier(n_jobs=-1)),
    ('Bagging Classifier', BaggingClassifier(n_jobs=-1)),
    ('Logistic Regression', LogisticRegression())
]

voting_ensemble = VotingClassifier(
    estimators=classifiers,
    n_jobs=-1,
    voting='soft'
)
voting_ensemble.fit(X_train, y_train)
y_pred_ensemble = voting_ensemble.predict(X_test)
# accuracy_score(y_test, y_pred_ensemble)

0.9617142857142857

# Stacking

In [96]:
X_train1, X_train2, y_train1, y_train2 = train_test_split(X_train, y_train, test_size=0.5)

classifiers_layer1 = {
    'random_forest': RandomForestClassifier(n_jobs=-1),
    'neural_network': MLPClassifier(),
    'extra_trees': ExtraTreesClassifier(n_jobs=-1)
}

for clf in classifiers_layer1:
    print(clf)
    classifiers_layer1[clf].fit(X_train1, y_train1)
    
    print('Accuracy on Test:')
    y_pred = classifiers_layer1[clf].predict(X_test)
    score = accuracy_score(y_test, y_pred)
    print(score)
    
X_train_layer2 = np.c_[[classifiers_layer1[clf].predict(X_train2) for clf in classifiers_layer1]].T

random_forest
Accuracy on Test:
0.9511428571428572
neural_network
Accuracy on Test:
0.9287857142857143
extra_trees
Accuracy on Test:
0.9540714285714286


In [97]:
classifier_layer2 = RandomForestClassifier(n_jobs=-1)
classifier_layer2.fit(X_train_layer2, y_train2)

RandomForestClassifier(n_jobs=-1)

In [99]:
X_test_layer2 = np.c_[[classifiers_layer1[clf].predict(X_test) for clf in classifiers_layer1]].T

y_pred = classifier_layer2.predict(X_test_layer2)
accuracy_score(y_pred, y_test)

#Nice score, better than random forest and neural net alones

0.9533571428571429